In [2]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
import csv

import shutil
import numpy as np
import geopy
import pygmt
import pyproj
from geopy.distance import geodesic
from geopy.distance import distance
from os import system as sys


path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"
subset = "C:/Users/19049/coding_files/data/subset.ts"
folder = "C:/Users/19049/coding_files/faults/"
sub_folder = "C:/Users/19049/coding_files/faults/wb"
wb_2b = "C:/Users/19049/coding_files/wb_2b/"

to_open = path


In [3]:
#13 faults that are included in the project that we want to make worlbuilder files of
# format: '*_fault',\n
faults_to_include = [
    'Panamint_Valley_fault',
    'Southern_Death_Valley_fault',
    'Black_Mountain_fault',
    'Owl_Lake_fault',
    'Hunter_Mountain_fault',
    'Ash_Hill_fault',
    'Independence_Sierra_Nevada_fault', #Gotta add the _fault identification bit cause of
    'Independence_fault',               #these two lines
    'Little_Lake_fault',
    'Airport_Lake_fault',
    'Eastern_Little_Lake_north_north_east_cross_fault',
    'Owens_Valley_fault',
    'Northern_Death_Valley_fault',
    #'-Casmalia_fault', #cause of this guy we gotta have the '-' at the beginning, not sure if this will bug the wb file, but i think it can be stripped later
    'thing'                             #if thing shows up in files, stuff is wack
    ]

In [4]:
#once features get cemented they will be added here for readability of main code
#makes an empty pandas dataframe to fill the adjusted values with
#",'Elevation(m)'"
def make_empty_fault_df():
    global fault_dataframe
    fault_dataframe = pd.DataFrame([],columns = ['Easting(m)','Northing(m)'])

#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the fault_dataframe dataframe with the elements adjusted
#add ", float_list[2]" to end of fault_dataframe to log the elevation
#fault_dataframe.loc[len(fault_dataframe)]
def adjust_to_df():
        string_line = line.split()
        float_list = list(map(float, string_line))
        fault_dataframe.loc[len(fault_dataframe)] = float_list[0]-345000, float_list[1]-3870000
        # print(bruh)
        
def adjust_to_wb():
        
        string_line = line.split()
        float_list = list(map(float, string_line))
        coors = [float_list[0]-345000, float_list[1]-3870000]
        coordinates.append(coors)  
        return coordinates
    
        

#takes the finished fault_dataframe df and names it the current fault trace, making it a csv    
def save_to_file():
    fault_dataframe.to_csv(folder+use_name+".csv")
def save_to_wb():
    wb.append(feature_string)

In [5]:
#tool to delete working file
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)

In [6]:
#Takes the raw data file and writes it to a wb file in one step

#clears working file so we don't stack
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)
        

#initial condidtions and intitiations of flags
wb=[]
model = '"model":"fault"'
version = '"version":"0.5"'
dip_point = '"dip point":[0,0]'
Associated_with_current_fault = True
coordinates = []
included_fault_tracker = []
global name

#determine last line
with open(to_open) as f:
    for line in f:
        pass
    global last_line
    last_line = line

#initialize the wb file
with open("wb_wip.txt", "a") as f:
    f.write('{')
    f.write('\n')
    f.write('\t')
    f.write(version)
    f.write(', \n')
    f.write('\t')
    f.write('"features":')
    f.write('\n')
    f.write('\t')
    f.write('[')
    f.write('\n')
    
with open(to_open) as f:
    for line in f:
        #purpose of this is that the last set of coordinates were not being outputted, since the output was dependent on a new fault being read in, so the above for loop finds the last line and this if statement outputs the stuff if it is the last line. Might be a stupid way of doing this but idk how else to do it. Isn't invoked in the full data read, but still needed for robustness.
        if line == last_line:
            #flags to let the script know the flow of what it's supposed to do- wrapping the coordinates associated with each fault name
            #this one is here to differentiate between being done with the current set of coordinates for a given fault or not, setting it to false means we're done with that set of coordinates and can send the wrapped set to the wb file
            #by wrapped I mean the coordinates for a fault are all together within a list and can be associated with the name and whatever else of the fault
            Associated_with_current_fault = False
            is_number = False
            while Associated_with_current_fault == False:
                #sorts faults that we want from ones we don't based on the fault tracker flag
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name                        
                        f.write('\t')
                        f.write('{')
                        f.write(model)
                        f.write(',')
                        f.write(codename)
                        f.write(',')
                        f.write(dip_point)
                        f.write(', \n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"coordinates":[')
                        f.write('\n')
                        f.write(', \n'.join(str(i) for i in coordinates))
                        f.write('\n')
                        f.write('],')
                        f.write('\n') #start of segment buisness
                        f.write('\t')
                        f.write('\t')
                        f.write('"segments":')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('[')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('\t')
                        f.write('{"length":29000,"thickness":[2000],"angle":[89.9]}')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"temperature models":[{"model":"uniform","temperature":273}],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"composition models":')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('[')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('\t')
                        f.write('{"model":"smooth","compositions":[0,1], "side distance fault center": 1000.0, "center fractions":[0.1,0.1], "side fractions":[0.0,0.0]}')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write(']') #end of segment buisness
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('},') #has ,
                        f.write('\n')
                        
                    #print(coordinates)
                break
        #the beginning of each new fault will have this as the starting characters of that line and every coordinate until the start of the next fault will be associated with it, 
        if line.startswith("> "):
            #flags to let the script know the flow of what it's supposed to do- wrapping the coordinates associated with each fault name
            Associated_with_current_fault = False
            is_number = False
            while Associated_with_current_fault == False:
                #sorts faults that we want from ones we don't based on the fault tracker flag
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name                        
                        f.write('\t')
                        f.write('{')
                        f.write(model)
                        f.write(',')
                        f.write(codename)
                        f.write(',')
                        f.write(dip_point)
                        f.write(', \n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"coordinates":[')
                        f.write('\n')
                        f.write(', \n'.join(str(i) for i in coordinates))
                        f.write('\n')
                        f.write('],')
                        f.write('\n') #start of segment buisness
                        f.write('\t')
                        f.write('\t')
                        f.write('"segments":')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('[')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('\t')
                        f.write('{"length":29000,"thickness":[2000],"angle":[89.9]}')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"temperature models":[{"model":"uniform","temperature":273}],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"composition models":')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('[')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('\t')
                        f.write('{"model":"smooth","compositions":[0,1], "side distance fault center": 1000.0, "center fractions":[0.1,0.1], "side fractions":[0.0,0.0]}')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write(']') #end of segment buisness
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('},') #has ,
                        f.write('\n')
                       
                    #print(coordinates)
                break
            #initiates the fault tracker list and resets it
            included_fault_tracker = []

            for real_name in faults_to_include:                            
                if real_name in line:           
                    #important for changing the fault name that is associated with the coordinates, initiated above as a global variable
                    name = real_name
                    #initiates and resets the coordinates variable in conjuntion with the name above
                    coordinates = []
                    print(name)
                    #if we have the name of one of the faults (from the faults_to_include list) we want in the real_name line then we add a yes to the tracker
                    included_fault_tracker.append('yes') 
                else:
                    #going to be a lot of no
                    included_fault_tracker.append('no')
            #print(included_fault_tracker)
            
        elif line.startswith("#"): 
            #this file has 15 '#'s at the beginning and I wanted to be explicit with what happens there
            pass  
        
        else:     
            #flag saying that we're in the coordinates portion of the fault
            is_number = True 
            
            while is_number == True:                        
                if 'yes' in included_fault_tracker: 
                    #print('sure')
                    #changes the raw coordinate data to a comma seperated, adjusted to study area, list of floats in a list. Goes line by line and appends each pair: format [[165833.0625, 173051.0], [165133.0, 172049.5],...
                    adjust_to_wb()
                else:                           
                    pass
                
                break
        #this is here to flag that we're not done with the coordinates of a particular fault yet, part of the wrapping process
        Associated_with_current_fault = True  

#instead of differentiating which fault is the last one, I just cut off the last comma
with open("wb_wip.txt", "rb+") as f:
    f.seek(-3, os.SEEK_END)
    f.truncate()
#after we do all that we close the wb file
with open("wb_wip.txt", "a") as f:
    f.write('\n')
    f.write('\t')
    f.write(']')
    f.write('\n')
    f.write('}')


Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


In [7]:
#makes files containing each fault data, doesn't need to be run but helps with debugging
with open(to_open, encoding='utf-8') as f:
    for line in f:                                          
        if line.startswith("> "):                           
            is_number = False                               
            included_fault_tracker = [] 
            
            for real_name in faults_to_include:                            
                if real_name in line:                       
                    print(real_name)
                    global use_name
                    use_name = real_name                                        
                    make_empty_fault_df()                   
                    included_fault_tracker.append('yes')     
                else:                                       
                    included_fault_tracker.append('no')     
        elif line.startswith("#"):                          
            pass                                            
        else:                                               
            is_number = True                                
            while is_number == True:                        
                if 'yes' in included_fault_tracker:         
                    adjust_to_df()                      
                    
                else:                           
                    pass
                break
            save_to_file()                                  

Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Casmalia_fault
Casmalia_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


In [25]:
print(last_line)

156687.5625   3877500.7500   -76.058300000
